In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
from sklearn.preprocessing import OrdinalEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error


### Load dữ liệu

In [7]:
dataset_path='./Housing.csv'
df=pd.read_csv(dataset_path)
df

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


## Xử lý dữ liệu categorical: 
### Để có thể thực hiện bất kì tính toán nào, ta cần phải số hóa toàn bộ giá trị trong bảng dữ liệu. Nhận thấy có một số cột trong bảng dữ liệu ### dạng string, ta sẽ phải đổi toàn bộ các giá trị này về dạng số. Đầu tiên, ta kiểm tra các cột có kiểu dữ liệu là Object như sau:

In [8]:
categorical_cols=df.select_dtypes(include=['object']).columns.to_list()
print(categorical_cols)

['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']


In [9]:
ordinal_encoder = OrdinalEncoder()
encoded_categorical_cols = ordinal_encoder . fit_transform (df[ categorical_cols ])
encoded_categorical_df = pd.DataFrame(encoded_categorical_cols ,columns = categorical_cols)
numerical_df = df. drop ( categorical_cols , axis =1)
encoded_df = pd.concat([ numerical_df , encoded_categorical_df ] , axis =1)

In [10]:
encoded_df

,price,area,bedrooms,bathrooms,stories,parking,mainroad,guestroom,basement,hotwaterheating,airconditioning,prefarea,furnishingstatus
0,13300000,7420,4,2,3,2,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1,12250000,8960,4,4,4,3,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,12250000,9960,3,2,2,2,1.0,0.0,1.0,0.0,0.0,1.0,1.0
3,12215000,7500,4,2,2,3,1.0,0.0,1.0,0.0,1.0,1.0,0.0
4,11410000,7420,4,1,2,2,1.0,1.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,2,1.0,0.0,1.0,0.0,0.0,0.0,2.0
541,1767150,2400,3,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
542,1750000,3620,2,1,1,0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
543,1750000,2910,3,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Chuẩn hóa bộ dữ liệu: 
### Để việc tính toán thuận lợi, ta tiến hành chuẩn hóa toàn bộ giá trị trong bộ dữ liệu sử dụng StandardScaler() như sau:

In [13]:
normalizer = StandardScaler()
dataset_arr = normalizer.fit_transform(encoded_df)
dataset_arr

array([[ 4.56636513,  1.04672629,  1.40341936, ...,  1.4726183 ,
         1.80494113, -1.40628573],
       [ 4.00448405,  1.75700953,  1.40341936, ...,  1.4726183 ,
        -0.55403469, -1.40628573],
       [ 4.00448405,  2.21823241,  0.04727831, ..., -0.67906259,
         1.80494113, -0.09166185],
       ...,
       [-1.61432675, -0.70592066, -1.30886273, ..., -0.67906259,
        -0.55403469,  1.22296203],
       [-1.61432675, -1.03338891,  0.04727831, ..., -0.67906259,
        -0.55403469, -1.40628573],
       [-1.61432675, -0.5998394 ,  0.04727831, ..., -0.67906259,
        -0.55403469,  1.22296203]])

*** 
Tách dữ liệu X, y: Khi đã hoàn tất các bước tiền xử lý, lúc này ta sẽ tách dữ liệu ban đầu thành
hai biến X, y đại diện cho các đặc trưng và nhãn. Nhận thấy cột đầu tiên, price, là nhãn của bộ
dữ liệu, ta sẽ tiến hành tách X, y ra như sau:
***

In [15]:
X, y = dataset_arr [: , 1:] , dataset_arr [: , 0]
X,y

(array([[ 1.04672629,  1.40341936,  1.42181174, ...,  1.4726183 ,
          1.80494113, -1.40628573],
        [ 1.75700953,  1.40341936,  5.40580863, ...,  1.4726183 ,
         -0.55403469, -1.40628573],
        [ 2.21823241,  0.04727831,  1.42181174, ..., -0.67906259,
          1.80494113, -0.09166185],
        ...,
        [-0.70592066, -1.30886273, -0.57018671, ..., -0.67906259,
         -0.55403469,  1.22296203],
        [-1.03338891,  0.04727831, -0.57018671, ..., -0.67906259,
         -0.55403469, -1.40628573],
        [-0.5998394 ,  0.04727831, -0.57018671, ..., -0.67906259,
         -0.55403469,  1.22296203]]),
 array([ 4.56636513e+00,  4.00448405e+00,  4.00448405e+00,  3.98575468e+00,
         3.55497918e+00,  3.25530927e+00,  2.88072189e+00,  2.88072189e+00,
         2.73088693e+00,  2.69342819e+00,  2.69342819e+00,  2.62974834e+00,
         2.43121702e+00,  2.39375829e+00,  2.39375829e+00,  2.31884081e+00,
         2.31884081e+00,  2.24392333e+00,  2.20646459e+00,  2.1877352

***
Chia tập dữ liệu train, val: Dựa vào bộ dữ liệu gốc, ta cần chia thành hai tập dữ liệu con,
một dùng cho việc huấn luyện mô hình và một cho việc đánh giá mô hình. Ở đây, ta sẽ chia theo
tỷ lệ 7:3 và tham số ngẫu nhiên random_state = 1:
***

In [16]:
test_size = 0.3
random_state = 1
is_shuffle = True
X_train , X_val , y_train , y_val = train_test_split (
X, y,
test_size = test_size ,
random_state = random_state ,
shuffle = is_shuffle)

## Huấn luyện mô hình: 
Ta thực hiện huấn luyện mô hình với bộ dữ liệu train. Để huấn luyện mô hình Random Forest, các bạn sẽ sử dụng RandomForestRegressor():

In [32]:
regressor = RandomForestRegressor(random_state = random_state)
regressor.fit( X_train , y_train )

RandomForestRegressor(random_state=1)

In [21]:
regressor = AdaBoostRegressor(random_state = random_state)
regressor.fit( X_train , y_train )

AdaBoostRegressor(random_state=1)

In [26]:
regressor = GradientBoostingRegressor(random_state = random_state)
regressor.fit( X_train , y_train )

GradientBoostingRegressor(random_state=1)

### Đánh giá mô hình: 
Để biết được mô hình đã huấn luyện có hoạt động tốt trên các mẫu dữ liệu
mới hay không, ta sẽ đánh giá thông qua tập val. Đầu tiên, ta cho mô hình đã huấn luyện thực
hiện dự đoán trên toàn bộ tập val:

In [33]:
y_pred = regressor.predict( X_val )
# y_pred

In [34]:
mae = mean_absolute_error (y_val , y_pred )
mse = mean_squared_error (y_val , y_pred )
print ('Evaluation results on validation set :')
print (f'Mean Absolute Error : { mae}')
print (f'Mean Squared Error : { mse}')

Evaluation results on validation set :
Mean Absolute Error : 0.46093873321571177
Mean Squared Error : 0.37944418523089524
